In [1]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("data/通联收银宝商户账户接入报文规范-商户账户接口.docx")

data = loader.load()

data

[Document(page_content='通联收银宝商户账户接入报文规范\n\n商户账户接口V1.0\n\n\n\n\n\n\n\n\n\n\n\n版本\n\n日期\n\n作者\n\n备注\n\nV1.0\n\n2018-08-01\n\n聂宗波\n\n创建\n\n\n\n2018-08-31\n\n李德坚\n\n修改转账单号长度为32\n\n需改所有收银宝交易单号trxid为18位\n\n增加fintime字段响应\n\n增加1001响应码\n\n\n\n\n目录\n\n\t1接口介绍\t3\n\n\t1.1功能介绍\t3\n\n\t1.2接入方式\t3\n\n\t1.3接入地址\t3\n\n\t1.4 Sign签名机制介绍\t3\n\n\t2接口定义\t4\n\n\t2.1商户余额转账\t4\n\n\t2.2账户接口查询接口\t5\n\n\t2.3商户账户余额查询\t6\n\n\t3附录\t7\n\n\t3．1交易返回码说明\t7\n\n\n\n\n\n\n\n\n1接口介绍\n\n1.1功能介绍\n\n\t商户通过接口对账户的协议内操作\n\n1.2接入方式\n\n\t商户可通过收银宝商户服务平台申请开通，并自行配置接入KEY，接入APPID(平台分配给商户的接入应用ID)。\n\n\t接口通过HTTPS方式提供，交易参数通过POST方式提交上来，返回结果均为JSON格式的数据报文。\n\n1.3接入地址\n\n\t接入测试参数：\n\n\t\t商户号：990440153996000\n\nAPPID：00000000\n\nKEY：43df939f1e7f5c6909b3f4b63f893994\n\n\n\n1.4 Sign签名机制介绍\n\n\t这里将定义生成sign 字符串的方法。\n\n签名字符串为除了sign之外的其他字段，加上分配的key的键值对.\n\n所有参与签名的字段，按字段名的ASCLL码从小到大排序后，使用URL的键值对的格式(即key1=value1&key2=value2)拼接成字符串string.\n\nsign=md5(string.getbyte("utf-8")).toUpperCase()\n\n如：\n\nappid=00000000,cusid =990440153996000,paytype =0,trxamt

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [3]:
from langchain import PromptTemplate

# 提示
template = """使用以下上下文片段回答最后的问题。
如果你不知道答案，只需说你不知道，不要试图编造一个答案。
最多使用三个句子，并尽量简洁地回答。
{context}
问题：{question}
有用的答案："""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# LLM
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Ollama(base_url="http://localhost:11434",
             model="llama2-chinese:7b-chat-q8_0",
             verbose=True,
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

# QA链
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [5]:
question = "请概括通联收银宝商户账户接入报文规范作者有哪几位？"
result = qa_chain({"query": question})


V1.0 2018-08-01 聂宗波创建。

V1.0 2018-08-31 李德坚修改转账单号长度为32，需改所有收银宝交易单号trxid为18位，增加fintime字段响应，增加1001响应码。

李德坚修改。
